In [1]:
import pandas as pd

In [2]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

#display(raw_data)

In [3]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [4]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [5]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

,MP,W,D,L,MW,ML,GF,GA,GD,Par%,W%,Pts,Form
Gameweek 18,,,,,,,,,,,,,
Zola,10,7,0,3,3,1,147,140,7,55,70,5.0,🔼 ⏫ ⏬ 🔼 🔼
Calz,13,8,0,5,4,1,204,166,38,72,61,4.5,⏬ 🔼 ⏫ 🔼 🔽
Schurrle,11,7,0,4,3,2,179,164,15,61,63,3.5,🔼 ⏫ ⏬ 🔽 🔼
Virlo,6,4,0,2,2,1,97,86,11,33,66,2.5,⏫ 🔼 🔽 ⏬ 🔼
Helmet,14,8,0,6,3,3,201,210,-9,77,57,2.0,🔼 ⏫ ⏬ 🔼 🔼
Wilf,13,7,0,6,2,3,186,191,-5,72,53,0.5,🔼 🔽 ⏬ 🔽 🔽
Keysey,6,3,0,3,1,1,87,78,9,33,50,0.0,🔽 🔼 🔽 ⏬ ⏫
Peggy,5,2,0,3,1,0,78,70,8,27,40,-0.5,🔽 🔽 🔼 ⏫ 🔽
Doug,10,4,0,6,3,2,140,136,4,55,40,-1.5,🔽 ⏫ 🔽 ⏬ 🔽


In [6]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [7]:
display(raw_data)

,Name,Date,GF,GA,Team,GD,GamePoints,W,D,L,MW,ML,Result,game_rank
0,Calz,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,13.0
1,Helmet,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,14.0
2,Kim,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,14.0
3,Mate,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,15.0
4,Peggy,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Helmet,2023-06-22,18,16,DARKS,2,1.0,1,0,0,0,0,🔼,1.0
173,Schurrle,2023-06-22,18,16,DARKS,2,1.0,1,0,0,0,0,🔼,1.0
175,Wilf,2023-06-22,16,18,BLANCOS,-2,-1.0,0,0,1,0,0,🔽,1.0
176,Calz,2023-06-22,16,18,BLANCOS,-2,-1.0,0,0,1,0,0,🔽,1.0


In [8]:
def count_same_team(df, name_col, date_col, team_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            print(same_team_names)
    return count

In [9]:
count_same_team(raw_data, 'Name', 'Date', 'Team')

['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Kim' 'Mate' 'Noel' 'Reaper']
['Kim

0